## Uitleg notebook 2: stap 2 aql toepassen en stap 3 statistieken maken

In [1]:
# import

import torch
import torchvision.models as models
import torch.nn as nn

import random

In [3]:
# model importeren

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

num_classes = 4

resnet = models.resnet18(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, num_classes)
# resnet = resnet.to(device)

# resnet = torch.load(resnet, "model_apple_resnet.pth")
# resnet.load_state_dict(torch.load("../models/resnet18_1024_weights.pth"))
# resnet.load_state_dict(torch.load("model_apple_resnet.pth"))
resnet.load_state_dict(torch.load('best_model_1.pth'))
resnet.eval()

cuda


/home/pans/miniconda3/envs/miw/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pans/miniconda3/envs/miw/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# test set aanbieden aan model en resultaten meten

# stel we bieden 500 samples aan, hoe is dan de verdeling over de vier klassen A,B,C,D

# vanaf hier kunnen we AQL berekenen obv 2 klassen: goed en niet-goed

# vanaf hier kunnen we AQL ook berekenen obv 4 klassen:
- 1: supermarkt kwaliteit. Komt overeen met klasse A/Healthy
- 2: appelmoes kwaliteit. Komt overeen met klasse B / beurs
- 3: stroop kwaliteit. Komt overeen met klasse C: stab
- 4: afgekeurd. Komt overeen met rotten.

Stel: 500 samples met:
classA_healthy = 350
classB_blotch = 70
classC_rot_ = 50
classD_scab = 30

In [55]:
# aql 

#----------------------------------------

# import random as rd

#----------------------------------------

# Define the params
lotSize = 500
sampleSize = 16

# Level is defined as percentage
aql = 15 / 100

#----------------------------------------

# No pun intended!
numBadApples = round(aql * lotSize)
numGoodApples = lotSize - numBadApples

print(f"Number of bad apples: {numBadApples}")
print(f"Number of good apples: {numGoodApples}")

applesBatch = []

# Append bad cups (encoded as 0)
for i in range(numBadApples):

    applesBatch.append(0)

# Append good cups (encoded as 1)
for i in range(numGoodApples):

    applesBatch.append(1)

#----------------------------------------

inspectionRounds = 10

results = []

for i in range(inspectionRounds):

    # Draw some random apples from the batch
    samples = random.sample(applesBatch, k = sampleSize)
    # kan je met sample ook twee keer dezelfde trekken? Check inbouwen?
    
    # Start fresh
    bads = 0

    # Count the number of bad apples
    for s in samples:

        # Bad apple?
        if s == 0:

            bads += 1

    # Store in results list
    results.append(bads)

#----------------------------------------

# Extract some statistics
avgBads = sum(results) / len(results)
maxBads = max(results)

print(f"The AVERAGE number of bad apples found is {avgBads}")
print(f"The MAXIMUM number of bad apples found is {maxBads}")

#----------------------------------------

Number of bad apples: 75
Number of good apples: 425
The AVERAGE number of bad apples found is 2.8
The MAXIMUM number of bad apples found is 6


In [ ]:
# statistieken